# Reddit Scraping
In order to scrape posts from Reddit, we used the [`praw` python library](https://github.com/praw-dev/praw). 
In order to use that, we need:
- a Reddit account.
- a Reddit app ([here](https://www.reddit.com/prefs/apps) you can create one). Here you'll find the client id under the app name once you create it, as well as the client secret and the user agent, which is your app name.

Once we have those, we can access reddit from the terminal using these steps(it is also possible to access as a user by adding username and account if you want to use it in write mode):

In [6]:
#pip install praw
import praw
reddit = praw.Reddit(client_id='CMO4Nf8Dpd3YE_lftmqnHg', client_secret='79YxaeUzQYqtVpvev6SbWbCMvF70-g', user_agent='little_digging')

We can analyse posts, post requirements etc. for a single subreddit or for all of them(by using the subreddit 'all'); we can also select top posts or do searches. 
# sistemare data utc

In [7]:
for i in reddit.subreddit('all').top(limit=5): #this will print the top posts all time
    print(i.title + ' Author: ' + i.author.name +  ' Link: https://www.reddit.com' + i.permalink + ' Subreddit: r/' + i.subreddit.display_name + ' Upvote Ratio: ' + str(i.upvote_ratio) + ' Date: '+str(i.created_utc))


I’ve found a few funny memories during lockdown. This is from my 1st tour in 89, backstage in Vegas. Author: ReallyRickAstley Link: https://www.reddit.com/r/pics/comments/haucpf/ive_found_a_few_funny_memories_during_lockdown/ Subreddit: r/pics Upvote Ratio: 0.99 Date: 1592410647.0
Times Square right now Author: SomeGuyInDeutschland Link: https://www.reddit.com/r/wallstreetbets/comments/l8rf4k/times_square_right_now/ Subreddit: r/wallstreetbets Upvote Ratio: 0.99 Date: 1612029638.0
Joe Biden elected president of the United States Author: throwawaynumber53 Link: https://www.reddit.com/r/news/comments/jptqj9/joe_biden_elected_president_of_the_united_states/ Subreddit: r/news Upvote Ratio: 0.88 Date: 1604766517.0
The Senate. Upvote this so that people see it when they Google "The Senate". Author: serventofgaben Link: https://www.reddit.com/r/movies/comments/62sjuh/the_senate_upvote_this_so_that_people_see_it_when/ Subreddit: r/movies Upvote Ratio: 0.96 Date: 1491051474.0
My cab driver toni

# Inserire spiegazione differenza tra le diverse scale hot top controversial
Now, for our analysis we will use data from the subreddit r/conspiracy and we will try to study the ramifications of the posts.

In [8]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

#put the subreddit already passed here
id_to_avoid = [reddit.subreddit('conspiracy').id]

for i in reddit.subreddit("conspiracy").hot(limit=5): #recupera i 5000 post hot di conspiracy
    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + l'url che viene postato


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]): #trova quando il post è stato citato
        if repost.subreddit_id not in id_to_avoid:
            
            subreddit_url = str(repost.subreddit)
            subreddit_url = "https://www.reddit.com/r/" + subreddit_url
            if subreddit_url in conspiracy_dict.keys():
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1][0] +=1
            else:
                conspiracy_dict[subreddit_url]=[[],[1]]
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_hot_url.csv',index=False)

An interesting thing we noted is that a good part of the propagation of news comes from bots, especially from the top posts.

In [9]:
'''
Da fare: script che accumula tutti gli id nella cartella
Let's suppose that the results are in "results/conspiracy_hot.csv". 
If you have multiple csv in the directory "my_directory", you may uncomment this and use the for loop in here:
import os
dir = "my_directory"
to_scan = [] #iterate over this
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and f[-4:] == ".csv":
        to_scan.append(f) 
id_to_analyse = []

for file in to_scan:
    df1 = pd.read_csv('results/conspiracy_hot.csv')
    for column in df1.columns:
        value = df1[column][1][1:-1]
        if int(value) >= 5:
            ind = column.index('/r/')
            id = column[ind+3:]
            id_to_analyse.append(id)

'''

df1 = pd.read_csv('results/conspiracy_hot.csv')
id_to_analyse = []
for column in df1.columns:
    value = df1[column][1][1:-1]
    if int(value) >= 5:
        ind = column.index('/r/')
        id = column[ind+3:]
        id_to_analyse.append(id)

for subreddit in id_to_analyse:
    post_list=list()
    subreddit_list = list()
    conspiracy_dict=dict() 
    for i in reddit.subreddit(subreddit).top(limit=500):
        post_list.append((i.title, i.score, "https://www.reddit.com"+i.permalink))


    for post in post_list:
        for repost in reddit.subreddit('all').search('url:'+post[2]):
            if repost.subreddit_id != subreddit: #cosa facciamo?
                subreddit_url = str(repost.subreddit)
                subreddit_url = "https://www.reddit.com/r/" + subreddit_url
                if subreddit_url in conspiracy_dict.keys():
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                    conspiracy_dict[subreddit_url][1][0] +=1
                else:
                    conspiracy_dict[subreddit_url]=[[],[1]]
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
    df = pd.DataFrame(conspiracy_dict)
    df.to_csv(r'results/'+subreddit+'.csv')
    

FileNotFoundError: [Errno 2] No such file or directory: 'results/conspiracy_hot.csv'

# Sentiment analysis
To do sentiment analysis, we will use the syuzhet package in R.
First we need to extract the comments inside a Reddit post. We will ignore the comments of bots, since they do not provide information about how the user interacts with it, even though it can interfere with it.
https://realpython.com/python-nltk-sentiment-analysis/
Caveat: \s indica il sarcasmo

In [10]:
import random
def iterSample(iterable, samplesize):
    results = []
    for i, v in enumerate(iterable):
        r = random.randint(0, i)
        if r < samplesize:
            if i < samplesize:
                results.insert(r, v) # add first samplesize items in random order
            else:
                results[r] = v # at a decreasing rate, replace random items

    if len(results) < samplesize:
        raise ValueError("Sample larger than population.")

    return results

In [11]:
top_comments_list = []
import pandas as pd
conspiracy_df = pd.read_csv('results/conspiracy_url_copy.csv', sep=';', encoding='unicode_escape')
for column in conspiracy_df.columns: # qui inserire un loop sui permalink
        list_post= conspiracy_df[column][1].split(',')
        for post_url in list_post:
                tmp = post_url
                
                while "'" in tmp:
                        tmp = tmp.replace("'","")
                while "[" in tmp:
                        tmp = tmp.replace("[","")
                while "]" in tmp:
                        tmp = tmp.replace(']','')
                try:
                        post = reddit.submission(url=tmp)
                except: # some posts may be removed
                        continue
                try:
                        post.comments.replace_more(limit=0)
                except:
                        print(f'{tmp} was not accessible')
                        continue
                result = []
                if len(post.comments)> 500:
                        for comment in iterSample(post.comments, 500):
                                result.append(comment)
                else:
                        for comment in post.comments:
                                result.append(comment)
                result={'permalink':tmp,'comments': result, 'target': conspiracy_df[column][0]}
                top_comments_list.append(result)


https://www.reddit.com/r/esist/comments/7hh4z8/i_was_just_handed_a_479page_tax_bill_a_few_hours/ was not accessible
https://www.reddit.com/r/AssholesGetBanned/comments/mqbfns/damn_this_one_was_too_easy/ was not accessible


# Sentiment Analysis


In [ ]:
import nltk
nltk.download(['stopwords', "vader_lexicon"])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [13]:
from nltk.sentiment import SentimentIntensityAnalyzer
from statistics import mean
sent = SentimentIntensityAnalyzer()
sentiment_values = dict()
for post in top_comments_list:
    post_sentiment = [] # median sentiment
    bot = 'I am a bot'
    
    for comment in post['comments']:
        body = comment.body
        if not bot in body:
            for reply in comment.replies:
                if not bot in reply.body:
                    body += reply.body
                for second_reply in reply.replies:
                    if not bot in second_reply.body:
                        body += second_reply.body
        val = sent.polarity_scores(body)['compound']
        if val < 0.01 or val > 0.01:
            post_sentiment.append(val)
    try:
        post_sentiment = mean(post_sentiment)
    except:
        post_sentiment = None
    sentiment_values[post['permalink']] =(post_sentiment, 'conspiracy', post['target'])

In [14]:
print(sentiment_values)

{'https://www.reddit.com/r/no_sob_story/comments/dr9b79/man/': (-0.5232666666666667, 'conspiracy', 'https://www.reddit.com/r/no_sob_story'), 'https://www.reddit.com/r/u_Spriggs555/comments/e2szmt/why_are_we_letting_this_happen/': (None, 'conspiracy', 'https://www.reddit.com/r/u_Spriggs555'), 'https://www.reddit.com/r/pepperidgefarmskarma/comments/dovpc2/reminder_80_days_ago_a_billionaire_pedophile/': (None, 'conspiracy', 'https://www.reddit.com/r/pepperidgefarmskarma'), 'https://www.reddit.com/r/u_the-big-gay-purple/comments/doupeo/reminder_80_days_ago_a_billionaire_pedophile/': (None, 'conspiracy', 'https://www.reddit.com/r/u_the-big-gay-purple'), 'https://www.reddit.com/r/KeepOurNetFree/comments/7gvq49/this_is_my_senator_he_sold_me_my_fellow/': (-0.1463242424242424, 'conspiracy', 'https://www.reddit.com/r/KeepOurNetFree'), 'https://www.reddit.com/r/BlueMidterm2018/comments/7gyw2a/this_is_my_senator_he_sold_me_my_fellow/': (0.0, 'conspiracy', 'https://www.reddit.com/r/BlueMidterm2018'